In [1]:
# --------------------- 
# 作者：lyhsdy 
# 来源：CSDN 
# 原文：https://blog.csdn.net/weixin_39416561/article/details/83926984 
# 版权声明：本文为博主原创文章，转载请附上博文链接！
#!coding=utf-8
import requests
import os
import re
import json
import datetime
import time
import pandas as pd
import numpy as np
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
# import win32api,win32con

In [2]:
def raw(text):  # 转化URL字符串
    escape_dict = {
            '/': '%252F',
            '?': '%253F',
            '=': '%253D',
            ':': '%253A',
            '&': '%26',
                   }
    new_string = ''
    for char in text:
        try:
            new_string += escape_dict[char]
        except KeyError:
            new_string += char
    return new_string
 
def mmm(item):
    item=raw(item)
    url='https://apapia.manmanbuy.com/ChromeWidgetServices/WidgetServices.ashx'
    s=requests.session()
    headers={
        'Host':'apapia.manmanbuy.com',
        'Content-Type':'application/x-www-form-urlencoded; charset=utf-8',
        'Proxy-Connection':'close',
        'Cookie':'ASP.NET_SessionId=uwhkmhd023ce0yx22jag2e0o; jjkcpnew111=cp46144734_1171363291_2017/11/25',
        'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_3 like Mac OS X) AppleWebKit/603.3.8 (KHTML, like Gecko) Mobile/14G60 mmbWebBrowse',
        'Content-Length':'457',
        'Accept-Encoding':'gzip',
        'Connection':'close',
    }
    postdata='c_devid=2C5039AF-99D0-4800-BC36-DEB3654D202C&username=&qs=true&c_engver=1.2.35&c_devtoken=&c_devmodel=iPhone%20SE&c_contype=wifi&' \
         't=1537348981671&c_win=w_320_h_568&p_url={}&' \
         'c_ostype=ios&jsoncallback=%3F&c_ctrl=w_search_trend0_f_content&methodName=getBiJiaInfo_wxsmall&c_devtype=phone&' \
         'jgzspic=no&c_operator=%E4%B8%AD%E5%9B%BD%E7%A7%BB%E5%8A%A8&c_appver=2.9.0&bj=false&c_dp=2&c_osver=10.3.3'.format(item)
    s.headers.update(headers)
    req=s.get(url=url,data=postdata,verify=False).text
 
    #print(req)
    try:
        js=json.loads(req)
#         print(js)
        title = js['single']['title']  ##名称
    except Exception as e:
        print(e)
        print("exit the function")
        return 0,0,0,0
    ###数据清洗
    jiagequshi=js['single']['jiagequshi']  ##价格趋势
    lowerPrice=js['single']['lowerPrice']  ##最低价格
    itemurl=js['single']['url']  ##商品链接
    date_list=[]   ##日期
    price_list=[]  ##价格
    ##日期转换   
    datalist=jiagequshi.replace('[Date.UTC(','').replace(')','').replace(']','').split(',')
    for i in range(0,len(datalist),4):
        if i !=0:
            day = int(datalist[i + 2])
            if int(datalist[i + 1]) == 12:
                mon = 1
                year = int(datalist[i]) + 1
            else:
                mon = int(datalist[i + 1]) + 1
                year = int(datalist[i])
            date = datetime.date(year=year, month=mon, day=day)
            date = date - datetime.timedelta(days=1)
            price = float(datalist[i -1])
            if date > datetime.date(2018, 10, 11) and date < datetime.date(2018, 12, 11):
                if date not in date_list:
                    date_list.append(date)
                    price_list.append(price)
 
        day=int(datalist[i + 2])
        if int(datalist[i+1])==12:
            mon=1
            year=int(datalist[i])+1
        else:
            mon=int(datalist[i+1])+1
            year = int(datalist[i])
 
        date=datetime.date(year=year,month=mon,day=day)
        price=float(datalist[i+3])
        
        if date > datetime.date(2018, 10, 10) and date < datetime.date(2018, 12, 12):
            if date not in date_list:
                date_list.append(date)
                price_list.append(price)

    data={'date':date_list,'price':price_list}
    df = pd.DataFrame(data)
#     df.to_csv('out.csv',index=False,mode='a',encoding="GB18030")  ##保存数据
    return title, lowerPrice, itemurl, df


In [3]:
def completeFullData(df):
    base = datetime.date(2018,12,11)
    numdays = 62
    date_list = [base - datetime.timedelta(days=x) for x in range(0, numdays)]
    data={'date':date_list}
    empty = pd.DataFrame(data)
    merged = empty.merge(df, on='date', how='outer', sort=True)
    if np.isnan(merged['price'].iloc[0]):
        merged.loc[0, 'price'] = merged[~merged['price'].isnull()]['price'].values[0]
    merged['price'].fillna(method='ffill', inplace=True)
    return merged

In [4]:
def append_output(file, title, lowerPrice, itemurl, full):
    row = []
    row.append(title)
    row.append(lowerPrice)
    row.append(itemurl)
    row.extend(full['price'].values.tolist())
    df = pd.DataFrame(row).T
    df.to_csv(file, mode='a', header=False, index=False, encoding="UTF-8")

In [13]:
# --------------------- 
# 作者：Understand_XZ 
# 来源：CSDN 
# 原文：https://blog.csdn.net/understand_XZ/article/details/80063285 
# 版权声明：本文为博主原创文章，转载请附上博文链接！
import urllib
import json
from urllib import parse,request
from bs4 import BeautifulSoup
 
# 根据页数和keywords采集内容
def get_content_from_keyword(keyword,page=1):
#     keyword_encode = parse.quote(keyword)  # unquote 解码
#     url = "https://search.jd.com/s_new.php?keyword=%s&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&wq=%s&page=%d&s=51&click=0"%(keyword_encode,keyword_encode,page)
#     r = request.urlopen(url)
    params = {'keyword':keyword,'page':page,'enc':'utf-8','area':1,'wq':keyword}
    data = parse.urlencode(params)
    opener = request.urlopen("https://search.jd.com/Search?"+data)
    content = opener.read()
    opener.close()
    return content # .decode('utf-8')


In [14]:
def get_item_list(keyword, page_num):
    cont = get_content_from_keyword(keyword, page=page_num)
    soup = BeautifulSoup(cont)
    goods_info = soup.select(".gl-item")
    good_ids = []
    for good_info in goods_info:
        data = {}
        good_info_dict = good_info.select(".p-name.p-name-type-2 a")[0].attrs
        good_id = good_info_dict['href'].split('/')[3][:-5]
        if good_id!='':
            good_ids.append(good_id)
    return good_ids

In [26]:
keywords = ['冰箱','洗衣机', '空调', '电视', '笔记本电脑', '手机']

In [27]:
for i in range(4,11):
    for keyword in keywords:
        print("page:%d"%(i))
        good_ids = get_item_list(keyword,i)
        for good_id in good_ids:
            print(good_id)
            item = 'http://item.jd.com/'+good_id+'.html'
            title, lowerPrice, itemurl, df = mmm(item)
            if itemurl==0:
                print("failed loading")
                time.sleep(630)
                continue
            print(title)
            if len(df)==0:
                print("cannot retrive data")
                continue
            full = completeFullData(df)
            append_output(keyword+'.csv', title, lowerPrice, itemurl, full)
            time.sleep(1)

page:4
6732123
TCL 215升 风冷无霜三门冰箱 AAT养鲜 变频风机 电脑控温 独立三温区 (典雅银) BCD-215TEWZ50
481584
奥马(Homa) 118升 双门冰箱 家用小型电冰箱 宿舍 租房 办公 迷你节能 PS6环保内胆 银色 BCD-118A5
100001923907
康佳(KONKA)383升薄壁技术 风冷无霜 对开门电冰箱 电脑温控 节能保鲜 两门家用 双开门 BCD-383WEGY5S
cannot retrive data
6116667
康佳(KONKA)400升 电脑温控 纤薄机身 静音节能 魔幻净味 对开门冰箱 (金色)BCD-400EGX5S
7176541
美的(Midea)421升多门冰箱 双变频 铂金净味 雷达感温 四门三温 风冷无霜 芙蓉金 BCD-421WTPM(E)
5675734
康佳（KONKA）206升 三门冰箱 家用小型 电冰箱 节能保鲜 三门三温 静音省电 （金色）BCD-206GX3S
7298179
容声(Ronshen) 576升 对开门冰箱 矢量双变频 大容量 风冷无霜 节能静音 电冰箱 BCD-576WD11HP
2784998
海尔(Haier)571升 双变频风冷无霜对开门冰箱  节能  智能WIFI 大容量 低温净味BCD-571WDEMU1
4315126
海尔（Haier）213升 小型三门冰箱 风冷无霜 DEO净味保鲜 007软冷冻 BCD-213WMPV
10016249137
上菱 (SHANGLING)BCD-183D 183升 双门冰箱小型 静音小冰箱 家用电冰箱双门 闪白银
4749462
美菱(MELING)209升三温区中门软冷冻 家用节能静音 三门冰箱 亚光银 BCD-209M3CX
7427648
创维(Skyworth)455升十字对开门冰箱 风冷无霜 电脑控温 分类存储(普利金) BCD-455WY
7321846
容声(Ronshen) 426升 十字对开门变频冰箱 纤薄嵌入 风冷无霜 静音节能 大冷冻力 BCD-426WD12FP
4580075
美的(Midea)228升 三门冰箱 变频无霜 日耗电0.6度 APP控制 智能电冰箱 爵士棕BCD-228WTPZM(E)
4103841
海信 (Hisense) 220升 三门电冰箱 中门

SSLError: HTTPSConnectionPool(host='apapia.manmanbuy.com', port=443): Max retries exceeded with url: /ChromeWidgetServices/WidgetServices.ashx (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [ ]:
import urllib.request
url = 'http://item.jd.com/3973386.html'
proxy_support = urllib.request.ProxyHandler({'http':'121.40.108.76'})
#参数是一个字典{'类型':'代理ip:端口号'}
opener = urllib.request.build_opener(proxy_support)
#定制opener
opener.add_handler=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36')]
#add_handler给加上伪装
urllib.request.install_opener(opener)


In [ ]:
urllib.request.urlopen('http://www.baidu.com')

In [ ]:
response = urllib.request.urlopen(url)

In [ ]:
response

In [18]:
item = 'http://item.jd.com/6559653.html'
title, lowerPrice, itemurl, df = mmm(item)
print(title)
full = completeFullData(df)

ConnectionError: HTTPSConnectionPool(host='apapia.manmanbuy.com', port=443): Max retries exceeded with url: /ChromeWidgetServices/WidgetServices.ashx (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11937c9e8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
import urllib.request as urllib2
url = "http://www.ip181.com/"
proxy_support = urllib2.ProxyHandler({'http':'121.40.108.76'})
#参数是一个字典{'类型':'代理ip:端口号'}
opener = urllib2.build_opener(proxy_support)
#定制opener
opener.add_handler=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36')]
#add_handler给加上伪装
urllib2.install_opener(opener)

In [ ]:
response = urllib2.urlopen(url)

print (response.read().decode('gbk'))

In [ ]:
# import urllib.request
# import urllib.error
 
# def use_proxy(url, proxy_addr):
#     proxy = urllib.request.ProxyHandler({'http': proxy_addr})
#     opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
#     urllib.request.install_opener(opener)
#     data = urllib.request.urlopen(url).read().decode('utf-8', 'ignore')
#     return data
 
# proxy_addr = '61.135.217.7:80'  # 可能已经失效，百度'代理ip'有免费的
# url = 'http://www.baidu.com'
# data = use_proxy(url, proxy_addr)
# print(len(data))